<a href="https://colab.research.google.com/github/Rayzon3/Natural-Language-Processing/blob/main/Sentiment_140_with_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 132kB/s 
     |████████████████████████████████| 983kB 51.2MB/s 
     |████████████████████████████████| 266kB 50.8MB/s 
     |████████████████████████████████| 1.3MB 53.9MB/s 
     |████████████████████████████████| 471kB 44.6MB/s 
     |████████████████████████████████| 1.1MB 46.1MB/s 
     |████████████████████████████████| 2.9MB 54.8MB/s 
     |████████████████████████████████| 890kB 38.8MB/s 
  Created wheel for ktrain: filename=ktrain-0.25.0-cp36-none-any.whl size=25274500 sha256=3b5397b17ca61c94232721995a356d61e70fd004349a13d1cd900b9338764545
  Stored in directory: /root/.cache/pip/wheels/5c/73/05/f36d0027bb6575384e21506dbba8db36a7825f15a24f09b2d5
  Created wheel for keras-bert: filename=keras_bert-0.86.0-cp36-none-any.whl size=34145 sha256=94656b3a65476ef0f1c4860e52ba0daffa948c7699947868c989df27fb0edf3f
  Stored in directory: /root/.cache/pip/wheels/66/f0/b1/748128b58562fc9e31b907bb5e2ab6a35eb37695e83911236b
  Created wheel for lan

In [2]:
import numpy as np
import pandas as pd
import re

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import transformers
import ktrain
from ktrain import text

In [3]:
data = pd.read_csv('/content/drive/My Drive/Dataset/sentiment140.csv', sep=',', header=None, encoding='latin')

In [4]:
data.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [5]:
labeled_data = data[[0, 5]]

In [6]:
labeled_data[0] = labeled_data[0].apply(lambda x: 0 if x == 0 else 1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [7]:
labeled_data.columns = ['label', 'text']

In [8]:
labeled_data.head()

,label,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [9]:
hashtags = re.compile(r"^#\S+|\s#\S+")
mentions = re.compile(r"^@\S+|\s@\S+")
urls = re.compile(r"https?://\S+")

def process_text(text):
  text = hashtags.sub(' hashtag', text)
  text = mentions.sub(' entity', text)
  return text.strip().lower()
  
def match_expr(pattern, string):
  return not pattern.search(string) == None

def remove_urls(dataset):
    link_with_urls = dataset.text.apply(lambda x: match_expr(urls, x))
    return dataset[[not e for e in link_with_urls]]

link_with_urls = labeled_data.text.apply(lambda x: match_expr(urls, x))

In [10]:
from sklearn.model_selection import train_test_split
TRAIN_SIZE = 0.20
VAL_SIZE = 0.05
dataset_count = len(labeled_data)

df_train_val, df_test = train_test_split(labeled_data, test_size=1-TRAIN_SIZE-VAL_SIZE, random_state=42)
df_train, df_val = train_test_split(df_train_val, test_size=VAL_SIZE / (VAL_SIZE + TRAIN_SIZE), random_state=42)

print("TRAIN size:", len(df_train))
print("VAL size:", len(df_val))
print("TEST size:", len(df_test))

TRAIN size: 320000
VAL size: 80000
TEST size: 1200000


In [11]:
df_train = remove_urls(df_train)

df_train.text = df_train.text.apply(process_text)
df_val.text = df_val.text.apply(process_text)
df_test.text = df_test.text.apply(process_text)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [12]:
df_train.head()

,label,text
1380904,1,strawberry/peanut butter m&amp;ms aren't gross...
1249515,1,please send me the house i want ! please?
322170,0,entity wish it was me but i just shamefully ha...
337708,0,trust me to be off college but have to do the ...
1474999,1,just got pokemon pearl. wanna ev train my star...


In [13]:
(X_train, y_train), (X_test, y_test), preprocess = text.texts_from_df(train_df=df_train,
                   text_column='text',
                   label_columns='label',
                   val_df=df_val,
                   maxlen=400,
                   preprocess_mode='bert',)

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [14]:
X_train[0].shape

(305949, 400)

In [15]:
#Making Model
model = text.text_classifier(name='bert',
                             train_data=(X_train, y_train),
                             preproc=preprocess)

Is Multi-Label? False
maxlen is 400
done.


In [16]:
#Learning Rate
learner = ktrain.get_learner(model=model,
                             train_data=(X_train, y_train),
                             val_data=(X_test, y_test),
                             batch_size=6)

In [17]:
#Training
learner.fit_onecycle(lr = 2e-5, epochs = 1)



begin training using onecycle policy with max lr of 2e-05...
19501/50992 [==========>...................] - ETA: 5:05:18 - loss: 0.3899 - accuracy: 0.8242

KeyboardInterrupt: ignored

In [18]:
learner

In [19]:
predictor = ktrain.get_predictor(learner.model, preprocess)

In [43]:
#Test 
data = ['Everything is going good!']

In [44]:
sentiment = predictor.predict(data)
if sentiment[0] is 'label':
  print('Positive')
else:
  print('Negative')

Positive


In [35]:
data = ["Today's day really sucked"]

In [41]:
sentiment = predictor.predict(data)
if sentiment[0] is 'label':
  print('Positive')
else:
  print('Negative')

Negative


In [45]:
predictor.save('bert')